# SVM Demo

The SVM loss function is given by:

$$ J =  J_w + C J_m $$

$$ J_w =  \frac{1}{2}\vec w\cdot \vec w  $$

$$ J_m =  \sum h_1( y_i p(x_i,w)) $$

The animation below allows you to play with the location of the demarcation line and the width of the margin to experiment their effect on the SVM loss.

$\theta$ is the angle between the line and the x axis, $d$ is the distance of the $z=0$ line to the origin.  




In [5]:
import bqplot as bq
import numpy as np
from bqplot import pyplot as plt
from ipywidgets import widgets
import sklearn.datasets

data_dic = sklearn.datasets.load_iris()
features = data_dic['data']
targets = data_dic['target']
c1 = features[targets==0]
c2 = features[targets==1]
ind1, ind2 = 0,1
def subSample(nData):
    X = np.empty((2*nData,2))
    X[:nData] = c1[:nData,:2]
    X[nData:] = c2[:nData,:2]
    Y = np.empty(2*nData)
    Y[:nData] = np.ones(nData)
    Y[nData:] = -np.ones(nData)
    return X,Y

X, Y = subSample(len(c1))
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

red = X[Y==1]
green = X[Y==-1]

ts = np.linspace(-10,10,100)
t = 0.0
d = 0.0
w = 1.0

def calculateXY(t, d, w):
    
    x = np.array([-10 * np.cos(t), 10 * np.cos(t)])
    y = np.array([-10 * np.sin(t), 10 * np.sin(t)])
    nx = - np.sin(t) 
    ny = np.cos(t) 
    x = x + d*nx
    y = y + d*ny
    x1 = x + nx/w
    y1 = y + ny/w
    x2 = x - nx/w
    y2 = y - ny/w
    w0 = - w*d
    w1 = w*nx
    w2 =w*ny

    zr = np.dot( np.array([w1, w2]), red.T ) + w0
    tr = - zr + 1
    h1r = np.maximum(tr,0)
    zg = np.dot( np.array([w1, w2]), green.T ) +w0
    tg =  zg + 1
    h1g = np.maximum(tg,0)
    Jmargin = sum(h1g)+ sum(h1r)
    Jw = 0.5 * (w1**2 + w2**2) 
    return x, y, x1, y1, x2, y2, w0, w1, w2, Jmargin, Jw

def update(arg):
    t = angle.value
    d = distance.value
    w = length.value
    x, y, x1, y1, x2, y2, w0, w1, w2, Jm, Jw = calculateXY(t, d, w)
    line.x, line.y = x, y    
    line1.x, line1.y = x1, y1    
    line2.x, line2.y = x2, y2    
    w0label.value = "{:.3f}".format(w0)
    w1label.value = "{:.3f}".format(w1)
    w2label.value = "{:.3f}".format(w2)
    Jmlabel.value = "{:.3f}".format(Jm)
    Jwlabel.value = "{:.3f}".format(Jw)

    
x, y, x1, y1, x2, y2, w0, w1, w2, Jm, Jw = calculateXY(t,d, w)

fig = plt.figure(min_aspect_ratio=1,max_aspect_ratio=1) 

line = plt.plot(x, y,'b-')
line1 = plt.plot(x1, y1,'r--')
line2 = plt.plot(x2, y2,'g--')
plt.xlim(-3,3)
plt.ylim(-3,3)

angle = widgets.FloatSlider(min=0,max=2*np.pi,value=t, description=r"$\theta$", step=0.02)
angle.observe(update, 'value')
distance = widgets.FloatSlider(min=-3,max=3,value=d, description="$d$", step =0.02)
distance.observe(update, 'value')
length = widgets.FloatSlider(min=0.1,max=10,value=w, description="$||w||$")
length.observe(update, 'value')


w0label = widgets.Label("{:.3f}".format(w0))
w1label = widgets.Label("{:.3f}".format(w1))
w2label = widgets.Label("{:.3f}".format(w2))
Jmlabel = widgets.Label("{:.3f}".format(Jm))
Jwlabel = widgets.Label("{:.3f}".format(Jw))

sc1 = plt.scatter(red[:,0], red[:,1], fmt='r')
sc2 = plt.scatter(green[:,0], green[:,1], fmt='r')
sc1.colors=['Red']
sc2.colors=['Green']

w0labelbox = widgets.HBox([widgets.Label("$w_0 = $"), w0label])
w1labelbox = widgets.HBox([widgets.Label("$w_1 = $"), w1label])
w2labelbox = widgets.HBox([widgets.Label("$w_2 = $"), w2label])
Jmlabelbox = widgets.HBox([widgets.Label("$J_m = $"), Jmlabel])
Jwlabelbox = widgets.HBox([widgets.Label("$J_w = $"), Jwlabel])


widgets.VBox([angle,distance, length, w0labelbox, w1labelbox, w2labelbox, Jmlabelbox, Jwlabelbox, fig])

